# Loss Landscapes on CIFAR

In [1]:
import os
import json
import copy
from pathlib import Path

import torch
from torch.utils.data import DataLoader

import models
import ops.tests as tests
import ops.datasets as datasets
import ops.loss_landscapes as lls

In [2]:
# config_path = "configs/cifar10_general.json"
config_path = "configs/cifar100_general.json"

with open(config_path) as f:
    args = json.load(f)
    print("args: \n", args)

args: 
 {'dataset': {'name': 'cifar100', 'root': '../data', 'mean': [0.5071, 0.4867, 0.4408], 'std': [0.2675, 0.2565, 0.2761]}, 'train': {'warmup_epochs': 1, 'epochs': 200, 'batch_size': 128}, 'val': {'batch_size': 256, 'n_ff': 1}, 'model': {'block': {}}, 'optim': {'name': 'SGD', 'lr': 0.1, 'momentum': 0.9, 'nesterov': False, 'weight_decay': 0.0005, 'scheduler': {'name': 'MultiStepLR', 'milestones': [60, 120, 160], 'gamma': 0.2}}, 'env': {}}


In [3]:
dataset_args = copy.deepcopy(args).get("dataset")
train_args = copy.deepcopy(args).get("train")
val_args = copy.deepcopy(args).get("val")
model_args = copy.deepcopy(args).get("model")
optim_args = copy.deepcopy(args).get("optim")
env_args = copy.deepcopy(args).get("env")

In [4]:
dataset_train, dataset_test = datasets.get_dataset(**dataset_args, download=True)
dataset_name = dataset_args["name"]
num_classes = len(dataset_train.classes)

dataset_train = DataLoader(dataset_train, 
                           shuffle=True, 
                           num_workers=train_args.get("num_workers", 4), 
                           batch_size=train_args.get("batch_size", 128))
dataset_test = DataLoader(dataset_test, 
                          num_workers=val_args.get("num_workers", 4), 
                          batch_size=val_args.get("batch_size", 128))

print("Train: %s, Test: %s, Classes: %s" % (
    len(dataset_train.dataset), 
    len(dataset_test.dataset), 
    num_classes
))

Files already downloaded and verified
Files already downloaded and verified
Train: 50000, Test: 10000, Classes: 100


## Model

In [5]:
# VGG
# name = "vgg_dnn_19"
# name = "vgg_dnn_smoothing_19"
# name = "vgg_mcdo_19"
# name = "vgg_mcdo_smoothing_19"

# ResNet
name = "resnet_dnn_18"
# name = "resnet_dnn_smoothing_18"
# name = "resnet_mcdo_18"
# name = "resnet_mcdo_smoothing_18"

# name = "resnet_dnn_50"
# name = "resnet_mcdo_50"
# name = "resnet_dnn_smoothing_50"
# name = "resnet_mcdo_smoothing_50"

# Preact ResNet
# name = "preresnet_dnn_50"
# name = "preresnet_mcdo_50"
# name = "preresnet_dnn_smoothing_50"
# name = "preresnet_mcdo_smoothing_50"

# ResNeXt
# name = "resnext_dnn_50"
# name = "resnext_mcdo_50"
# name = "resnext_dnn_smoothing_50"
# name = "resnext_mcdo_smoothing_50"

# WideResNet
# name = "wideresnet_dnn_50"
# name = "wideresnet_mcdo_50"
# name = "wideresnet_dnn_smoothing_50"
# name = "wideresnet_mcdo_smoothing_50"


uid = ""  # Model UID required
model = models.get_model(name, num_classes=num_classes, tiny=True)
models.load(model, dataset_name, uid)

model: resnet_dnn_18 , params: 11.2M, output: [3, 100]


## Investigate the Loss Landscape

In [ ]:
gpu = torch.cuda.is_available()

model = model.cuda() if gpu else model.cpu()
metrics_list = []
for n_ff in [1]:
    print("N: %s, " % n_ff, end="")
    *metrics, cal_diag = tests.test(model, n_ff, dataset_test, verbose=False, gpu=gpu)
    metrics_list.append([n_ff, *metrics])

In [ ]:
scale = 1e-1
n = 21
metrics_grid = lls.get_loss_landscape(
    model, 1, dataset_train, 
    x_min=-1.0 * scale, x_max=1.0 * scale, n_x=n, y_min=-1.0 * scale, y_max=1.0 * scale, n_y=n,
)
leaderboard_path = os.path.join("leaderboard", "logs", dataset_name, model.name)
Path(leaderboard_path).mkdir(parents=True, exist_ok=True)
metrics_dir = os.path.join(leaderboard_path, "%s_%s_%s_x%s_losslandscape.csv" % (dataset_name, model.name, uid, int(1 / scale)))
metrics_list = [[*grid, *metrics] for grid, metrics in metrics_grid.items()]
tests.save_metrics(metrics_dir, metrics_list)